In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [4]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [5]:
## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

In [6]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [13]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify= dataset['Exited'])

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense        # To create Hidden layers.
from keras.layers import LeakyReLU,PReLU,ELU       # Activaction Funcitons
from keras.layers import Dropout      # Regularization Parameter

In [47]:
classifier = Sequential()

In [48]:
classifier.add(Dense(units= 10, activation= 'relu', kernel_initializer= 'he_normal', input_dim= 11))

In [51]:
classifier.add(Dropout(0.4))

In [52]:
classifier.add(Dense(units= 15, activation= 'relu', kernel_initializer= 'he_normal'))

In [54]:
classifier.add(Dropout(0.3))

In [55]:
classifier.add(Dense(units= 20, activation= 'relu', kernel_initializer= 'he_normal'))

In [56]:
classifier.add(Dropout(0.3))

In [57]:
classifier.add(Dense(units= 1, activation= 'sigmoid', kernel_initializer= 'glorot_uniform'))

In [58]:
classifier.compile(optimizer='Adamax', loss= 'binary_crossentropy', metrics=['accuracy'])

In [59]:
model_history= classifier.fit(X_train, y_train, validation_split=0.3, batch_size=10, epochs= 100)

Epoch 1/100
560/560 [==============================] - 1s 3ms/step - loss: 11088.9854 - accuracy: 0.5973 - val_loss: 2474.0386 - val_accuracy: 0.8062
Epoch 2/100
560/560 [==============================] - 1s 2ms/step - loss: 4247.3037 - accuracy: 0.6227 - val_loss: 962.5555 - val_accuracy: 0.8062
Epoch 3/100
560/560 [==============================] - 1s 2ms/step - loss: 1962.7522 - accuracy: 0.6495 - val_loss: 324.2708 - val_accuracy: 0.8062
Epoch 4/100
560/560 [==============================] - 1s 2ms/step - loss: 839.9996 - accuracy: 0.6598 - val_loss: 140.9569 - val_accuracy: 0.8062
Epoch 5/100
560/560 [==============================] - 1s 3ms/step - loss: 371.1320 - accuracy: 0.6796 - val_loss: 48.4412 - val_accuracy: 0.8062
Epoch 6/100
560/560 [==============================] - 1s 2ms/step - loss: 124.5314 - accuracy: 0.7084 - val_loss: 9.3413 - val_accuracy: 0.8062
Epoch 7/100
560/560 [==============================] - 1s 2ms/step - loss: 44.4106 - accuracy: 0.7530 - val_loss: 3.

Epoch 56/100
560/560 [==============================] - 2s 3ms/step - loss: 0.5118 - accuracy: 0.7920 - val_loss: 0.4941 - val_accuracy: 0.8062
Epoch 57/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5111 - accuracy: 0.7920 - val_loss: 0.4939 - val_accuracy: 0.8062
Epoch 58/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5123 - accuracy: 0.7920 - val_loss: 0.4937 - val_accuracy: 0.8062
Epoch 59/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5115 - accuracy: 0.7920 - val_loss: 0.4936 - val_accuracy: 0.8062
Epoch 60/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5111 - accuracy: 0.7918 - val_loss: 0.4936 - val_accuracy: 0.8062
Epoch 61/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5123 - accuracy: 0.7920 - val_loss: 0.4935 - val_accuracy: 0.8062
Epoch 62/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5118 - accuracy: 0.7920 - val_loss: 0.4933 - val_accuracy:

In [60]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [61]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1593,    0],
       [ 407,    0]], dtype=int64)

In [62]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
score

0.7965

# HyperParamter Tuning

In [63]:
## Perform Hyperparameter Optimization

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [64]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [65]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:        # This is the input layer. The first layer.
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
      
    # Last Layer
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [66]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [69]:
layers = [(20,), (40, 20), (45, 30, 15)]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [70]:
grid_result = grid.fit(X_train, y_train)

In [71]:
[grid_result.best_score_,grid_result.best_params_]

[0.7962499856948853,
 {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': (20,)}]

In [76]:
# Predicting the Test set results
y_pred = grid.predict(X_test)
y_pred = (y_pred > 0.5)

In [77]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1593,    0],
       [ 407,    0]], dtype=int64)

In [78]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
score

0.7965